In [88]:
class Node:
    def __init__(self, symbol, freq ,left=None, right=None):
        
        self.freq = freq  # Internal node: value is the frequency
        self.symbol = symbol    # Leaf node: character
        self.left = left
        self.right = right        
        
        
        # tree direction 0 or 1 
        self.huff = ''\
        
    #comparator, lt
    def __lt__(self, other):
        return self.freq < other.freq
    
    #tostring   
    def __str__(self):
        return '(' + str(self.symbol)+','+str(self.freq) + ',' + str(self.huff) + ')'
    
    
# class HierarchicalSoftmax:
    
#     def __init__(self, huffman_tree):
#         self.huffman_tree = huffman_tree

#     def predict(self, context):
#         node = self.huffman_tree
#         probabilities = []

#         while node:
#             if node.char:
#                 probabilities.append((node.char, 1.0))
#                 node = self.huffman_tree  # Reset to the root for the next prediction
#             else:
#                 left_prob = 1.0 / (1.0 + 2.0**(-node.left.value))
#                 right_prob = 1.0 - left_prob
#                 if context & 1:  # Check the least significant bit
#                     node = node.right
#                     probabilities.append((node.char, right_prob))
#                 else:
#                     node = node.left
#                     probabilities.append((node.char, left_prob))
#                 context >>= 1  # Right shift the context

#         return probabilities

In [127]:
import heapq # stdlib, basically the huffman tree.

#returns root of the tree.
def build_tree(freq_dict):
    
    nodes = []
    
    for k,v in freq_dict.items():

        heapq.heappush(nodes, Node(k, v))
    
    while len(nodes) > 1:
  
        # sort all the nodes in ascending order 
        # based on their frequency 
        left = heapq.heappop(nodes)
        right = heapq.heappop(nodes)

        # assign directional value to these nodes 
        #least
        
        left.huff = '0'
        #2nd least
        right.huff = '1'
        
        # combine the 2 smallest nodes to create 
        # new node as their parent 
        newNode = Node(left.symbol+right.symbol,left.freq+right.freq, left, right)
        
        heapq.heappush(nodes, newNode)
        
    return nodes[0]

In [128]:
# utility function for printing
def in_order_traversal(node,path):
    
    if node.left != None:
        
        in_order_traversal(node.left,path + node.left.huff)
    
    print(node)
    print(path)
    
    if node.right != None:
  
        in_order_traversal(node.right,path + node.huff)
    

In [129]:
# Example usage
if __name__ == "__main__":

    ft = {
        
        'c':63,
        'a':27,
        'z':72,
        'b':17,
        'y':89
    }
 
    root = build_tree(ft)
    print('root is : ', root)
    in_order_traversal(root,'')
#     traverse_tree(root)
    
#     for node in huffman_tree:
        
#         print(node)
    
    
    
    
    
#     hierarchical_softmax = HierarchicalSoftmax(huffman_tree)

#     # Example context (represented as an integer)
#     context = 0b10

#     # Predict probabilities for the context
#     probabilities = hierarchical_softmax.predict(context)

#     # Print the predicted probabilities
#     for char, prob in probabilities:
#         print(f"'{char}': {prob:.4f}")


root is :  (baczy,268,)
(b,17,0)
000
(ba,44,0)
00
(a,27,1)
000
(bac,107,0)
0
(c,63,1)
00
(baczy,268,)

(z,72,0)
0
(zy,161,1)

(y,89,1)
1
